In [69]:
import pandas as pd
import numpy as np
# from bs4 import BeautifulSoup
import re

In [45]:
# get list of urls
# draft_url_list = []
draft_years = range(1976,2025)
draft_url_list = ['https://en.wikipedia.org/wiki/%i_NBA_draft'%year for year in draft_years]
draft_url_list[:2]

['https://en.wikipedia.org/wiki/1976_NBA_draft',
 'https://en.wikipedia.org/wiki/1977_NBA_draft']

In [53]:
# get tables from each URL
table_list = []

rename_cols = {
    'Nationality[n 1]': 'Nationality',
    'Nationality [n 1]': 'Nationality',
    'Pos.': 'Position',
    'NBA Team': 'Team',
    'NBA team': 'Team',
    'School/Club team': 'School/Club Team',
    'School/club team': 'School/Club Team',
    'School / club team': 'School/Club Team',
    'School or club team': 'School/Club Team',
    '.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Rnd.': 'Round',
    '.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Round': 'Round',
}
drop_cols = ['External videos']

for index,draft_url in enumerate(draft_url_list):
    # print(index,draft_url)
    # fourth table is actual draft
    table_idx = 3
    if index == len(draft_url_list) - 1:
        table_idx = 2
    table_df = pd.read_html(draft_url)[table_idx]
    
    # add year
    table_df.insert(0,'Year', draft_years[index])

    # rename certain columns
    existing_rename_cols = {old: new for old, new in rename_cols.items() if old in table_df.columns}
    table_df.rename(columns=existing_rename_cols, inplace=True)

    # drop column if exist
    existing_drop_cols = [col for col in drop_cols if col in table_df.columns]
    table_df.drop(columns=existing_drop_cols, inplace=True)

    table_list.append(table_df)
table_list[0]

,Year,Round,Pick,Player,Position,Nationality,Team,School/Club Team
0,1976,1,1,John Lucas,G,United States,Houston Rockets (from Atlanta).mw-parser-outpu...,Maryland (Sr.)
1,1976,1,2,Scott May,F,United States,Chicago Bulls,Indiana (Sr.)
2,1976,1,3,Richard Washington,F/C,United States,Kansas City Kings,UCLA (Jr.)
3,1976,1,4,Leon Douglas,F/C,United States,Detroit Pistons,Alabama (Sr.)
4,1976,1,5,Wally Walker,F,United States,Portland Trail Blazers,Virginia (Sr.)
...,...,...,...,...,...,...,...,...
168,1976,10,169,Tim Stokes#,G,United States,Buffalo Braves,Canisius (Sr.)
169,1976,10,170,Mike Buescher#,F,United States,Washington Bullets,Seton Hall (Sr.)
170,1976,10,171,Elisha McSweeney#,F,Bahamas,Philadelphia 76ers,Minnesota State (Sr.)
171,1976,10,172,Otho Tucker#,F,United States,Boston Celtics,Illinois (Sr.)


In [54]:
combined_table_df = pd.concat(table_list, axis=0, ignore_index=True)
combined_table_df

,Year,Round,Pick,Player,Position,Nationality,Team,School/Club Team,0,1
0,1976,1.0,1,John Lucas,G,United States,Houston Rockets (from Atlanta).mw-parser-outpu...,Maryland (Sr.),NaN,NaN
1,1976,1.0,2,Scott May,F,United States,Chicago Bulls,Indiana (Sr.),NaN,NaN
2,1976,1.0,3,Richard Washington,F/C,United States,Kansas City Kings,UCLA (Jr.),NaN,NaN
3,1976,1.0,4,Leon Douglas,F/C,United States,Detroit Pistons,Alabama (Sr.),NaN,NaN
4,1976,1.0,5,Wally Walker,F,United States,Portland Trail Blazers,Virginia (Sr.),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4367,2024,2.0,55,Bronny James,SG,United States,Los Angeles Lakers (from L.A. Clippers)[AK],USC (Fr.),NaN,NaN
4368,2024,2.0,56,Kevin McCullar Jr.,SG,United States,Denver Nuggets (from Minnesota via Oklahoma Ci...,Kansas (Sr.),NaN,NaN
4369,2024,2.0,57,Ulrich Chomche,PF/C,Cameroon,Memphis Grizzlies (from Oklahoma City to Atlan...,APR BBC (Rwanda),NaN,NaN
4370,2024,2.0,Phoenix Suns (from Denver via Orlando;[AK][AN]...,Phoenix Suns (from Denver via Orlando;[AK][AN]...,Phoenix Suns (from Denver via Orlando;[AK][AN]...,Phoenix Suns (from Denver via Orlando;[AK][AN]...,Phoenix Suns (from Denver via Orlando;[AK][AN]...,Phoenix Suns (from Denver via Orlando;[AK][AN]...,NaN,NaN


### drop columns

In [61]:
# drop columns 0 and 1
combined_table_df.drop(columns=[0,1], inplace=True)
# drop rows from pick where non-numeric
combined_table_df['Pick'] = pd.to_numeric(combined_table_df['Pick'], errors='coerce')
combined_table_df.dropna(subset=['Pick'], inplace=True)

### create new columns

In [67]:
# create accolades columns, splitting player
# ^ = hall of famer | * = all nba + all star | x = all nba, + = all star, 
# ‡ = all nba, all star, ROTY | ~ = ROTY, # = never appeared in NBA game

accolades = {
    'HallOfFamer': r'\^',
    'AllNBAAllStar': r'\*',
    'AllNBA': r'x',
    'AllStar': r'\+',
    'AllNBAAllStarROTY': r'‡',
    'ROTY': r'~',
    'NeverPlayed': r'#'
}

# add accolade dummy column
for accolade, symbol in accolades.items():
    combined_table_df[accolade] = combined_table_df['Player'].apply(lambda x: 1 if re.search(symbol, x) else 0)

combined_table_df['Player'] = combined_table_df['Player'].str.replace(r'[\^*\+x‡~#]', '', regex=True).str.strip()

In [132]:
# split school year/league/country from School/Club Team
combined_table_df[['School/Club Team', 'School Year/League/Country']] = combined_table_df['School/Club Team'].fillna('').str.split('(', n=1, expand=True)
combined_table_df['School/Club Team'] = combined_table_df['School/Club Team'].str.strip()
combined_table_df['School Year/League/Country'] = combined_table_df['School Year/League/Country'].str.strip().str.rstrip(')')

# # split pick trade from team
combined_table_df[['Team', 'Pick Details']] = combined_table_df['Team'].str.split('(', expand=True,n=1)
combined_table_df['Team'] = combined_table_df['Team'].str.strip().str.split('[',expand=True)[0]
combined_table_df['Pick Details'] = combined_table_df['Pick Details'].str.split(')',expand=True)[0]

# # split Nationality
combined_table_df['Nationality'] = combined_table_df['Nationality'].str.split('[',expand=True)[0]
# combined_table_df[['Nationality','Nationality2']] = combined_table_df['Nationality'].str.split(r' +',expand=True)


,0,1,2,3,4
0,United,States,None,None,None
1,United,States,None,None,None
2,United,States,None,None,None
3,United,States,None,None,None
4,United,States,None,None,None
...,...,...,...,...,...
4366,United,States,None,None,None
4367,United,States,None,None,None
4368,United,States,None,None,None
4369,Cameroon,None,None,None,None


In [126]:
combined_table_df.head()

,Year,Round,Pick,Player,Position,Nationality,Team,School/Club Team,HallOfFamer,AllNBAAllStar,AllNBA,AllStar,AllNBAAllStarROTY,ROTY,NeverPlayed,School Year/League/Country,Pick Details
0,1976,1.0,1.0,John Lucas,G,United States,Houston Rockets,Maryland,0,0,0,0,0,0,0,Sr.,from Atlanta
1,1976,1.0,2.0,Scott May,F,United States,Chicago Bulls,Indiana,0,0,0,0,0,0,0,Sr.,None
2,1976,1.0,3.0,Richard Washington,F/C,United States,Kansas City Kings,UCLA,0,0,0,0,0,0,0,Jr.,None
3,1976,1.0,4.0,Leon Douglas,F/C,United States,Detroit Pistons,Alabama,0,0,0,0,0,0,0,Sr.,None
4,1976,1.0,5.0,Wally Walker,F,United States,Portland Trail Blazers,Virginia,0,0,0,0,0,0,0,Sr.,None


In [70]:
np.mean(combined_table_df.NeverPlayed) # 41% of drafted players never play an NBA game

0.4105504587155963

In [130]:
# export to csv
combined_table_df.to_csv('nba_drafts.csv', index=False)